In [1]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

train_data = pd.read_csv("dev/train_features.csv")
train_metadata = pd.read_csv("dev/train_metadata.csv")
ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ana)
ana.plot_data()


MachineLearningAnalyses 
  variables: 20 rows, 17 columns
  metadata: 20 rows, 14 columns



In [2]:
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="MaxAbsScaler")
iso = MachineLearningMethodIsolationForestSklearn(random_state = 30, contamination = "auto")

#scales and fits the model and upgrades analyses class to IsolationForestAnalyses
ana = scl.run(ana)
ana = iso.run(ana)
print(ana.__class__)

ana.plot_data()

<class 'src.StreamPort.machine_learning.analyses.IsolationForestAnalyses'>


c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [3]:
ana.train()
fig_train_scores=ana.plot_scores()
fig_train_scores.write_image("dev/figures/fig_train_scores.png", width=1100, height= 350, scale = 3)
fig_train_scores.show()

In [4]:
test_data = pd.read_csv("dev/test_features.csv")
test_metadata = pd.read_csv("dev/test_metadata.csv")

ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)

fig_test_scores = ana.plot_scores()
fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1100, height= 350, scale = 3)
fig_test_scores.show()

    index  threshold     score  confidence    class
0     116  -0.079893 -0.133674        1.67  outlier
1     117  -0.079893 -0.049327        0.62   normal
2     118  -0.079893 -0.012798        0.16   normal
3     119  -0.079893 -0.065803        0.82   normal
4     120  -0.079893 -0.016794        0.21   normal
5     121  -0.079893 -0.033935        0.42   normal
6     122  -0.079893 -0.016786        0.21   normal
7     123  -0.079893 -0.037751        0.47   normal
8     124  -0.079893 -0.059363        0.74   normal
9     125  -0.079893 -0.025975        0.33   normal
10    126  -0.079893 -0.033824        0.42   normal
11    127  -0.079893 -0.064599        0.81   normal
12    128  -0.079893 -0.046139        0.58   normal
13    129  -0.079893 -0.018531        0.23   normal
14    130  -0.079893 -0.007183        0.09   normal
15    131  -0.079893 -0.032575        0.41   normal
16    132  -0.079893 -0.037597        0.47   normal


Addition of new training data based on previous prediction results

In [5]:
#ana.add_prediction()
#print(ana)

In [6]:
fig_test_features = ana.plot_data()
fig_test_features.write_image("dev/figures/fig_test_features.png", width=1100, height= 350, scale = 3)
fig_test_features.show()

In [7]:
test2_data = pd.read_csv("dev/test2_features.csv")
test2_metadata = pd.read_csv("dev/test2_metadata.csv")
test_samples = test2_metadata["index"]

ana.predict(test2_data, test2_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
ana.plot_scores()

   index  threshold     score  confidence    class
0    266  -0.079893 -0.095691        1.20  outlier
1     47  -0.079893 -0.103143        1.29  outlier
2     17  -0.079893 -0.143523        1.80  outlier
3    119  -0.079893 -0.065803        0.82   normal
4     16  -0.079893 -0.146793        1.84  outlier


In [8]:
#ana.add_prediction()
#print(ana)

In [9]:
fig_test2_features = ana.plot_data()
fig_test2_features.write_image("dev/figures/fig_test2_features.png", width=1100, height= 350, scale = 3)
fig_test2_features.show()

# Collect IForest test records to label known/identified anomalies and pass to KNN

In [10]:
# join old train and test data with labels to be passed into knn
old_train_data = pd.read_csv("dev/train_features.csv")
old_train_metadata = pd.read_csv("dev/train_metadata.csv")

old_train_metadata["label"] = "normal"
metadata_columns = old_train_metadata.columns

old_train_data = pd.concat([old_train_metadata, old_train_data], axis = 1)
print("Normal data: ", old_train_data[metadata_columns]) #to be labelled "normal" as part of next stage

Normal data:      index             name                                               path  \
0      18  210812_Pac--005  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
1      19  210812_Pac--006  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
2      20  210812_Pac--007  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
3      21  210812_Pac--008  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
4      22  210812_Pac--009  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
5      23  210812_Pac--010  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
6      24  210812_Pac--011  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
7      25  210812_Pac--012  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
8      26  210812_Pac--013  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
9      27  210812_Pac--014  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
10     51  210813_Pac--005  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
11     52  210

In [11]:
import os

test_files = []
test_meta = []

#collect test records from IForest
path_to_test_records = "dev/test_record.csv"

test_record = pd.read_csv(path_to_test_records) if os.path.exists(path_to_test_records) else None
test_record = test_record.sort_values("date") if test_record is not None else None

if test_record is not None:
    result_logs = []
    for date in test_record["date"]:

        filepath = f"dev/test_{date}_classified_samples.csv"
        result_logs.append(filepath) if os.path.exists(filepath) else print(f"No records for {date}")

        featurepath = f"dev/test_{date}_features.csv"
        test_files.append(featurepath) if os.path.exists(featurepath) else print(f"No records for {date}")
        
        metapath = f"dev/test_{date}_metadata.csv"
        test_meta.append(metapath) if os.path.exists(metapath) else print(f"No records for {date}")
        
else:
    print("Not enough evidence of true inliers! Please run more tests")

In [12]:
summary = pd.DataFrame()
data = pd.DataFrame()
metadata = pd.DataFrame()

for i in range(len(test_files)):
    feature_df = pd.read_csv(test_files[i])
    data = pd.concat([data, feature_df], ignore_index=True)

    log = pd.read_csv(result_logs[i])
    summary = pd.concat([summary, log], ignore_index=True)

    meta = pd.read_csv(test_meta[i])
    metadata = pd.concat([metadata, meta], ignore_index=True)

#data from test records are newly labelled. Classification results over tests will be averaged 
new_data = pd.concat([metadata, data], axis = 1)

new_data = new_data.sort_values("index")
new_data.drop_duplicates(subset = "index", inplace=True)
new_data.reset_index(drop=True, inplace=True)

summary = summary.sort_values("index")
summary.reset_index(drop=True, inplace = True)
print("Test summary: ", summary.head)
summary.to_csv("dev/test_summary.csv", index=False)

Test summary:  <bound method NDFrame.head of     index  threshold     score  confidence    class  \
0      14  -0.080903 -0.111895        1.38  outlier   
1      47  -0.061153 -0.093758        1.53  outlier   
2      47  -0.061153 -0.093758        1.53  outlier   
3      49  -0.060696 -0.070741        1.17  outlier   
4      50  -0.061153 -0.016271        0.27   normal   
..    ...        ...       ...         ...      ...   
65    272  -0.061153 -0.038800        0.63   normal   
66    272  -0.061153 -0.038800        0.63   normal   
67    275  -0.061153 -0.037287        0.61   normal   
68    275  -0.061689 -0.061361        0.99   normal   
69    278  -0.061153 -0.051743        0.85   normal   

                          date  
0   2025-07-30 10-59-59-838221  
1   2025-07-29 16-35-41-271047  
2   2025-07-29 16-35-46-848140  
3   2025-07-29 16-40-22-832458  
4   2025-07-29 16-35-41-271047  
..                         ...  
65  2025-07-29 16-35-35-763234  
66  2025-07-29 16-35-46-848140

In [13]:
# EvaluateModelStability also prepares data for transfer from unsupervised to supervised learning by assigning true class labels
from src.StreamPort.machine_learning.methods import MachineLearningEvaluateModelStabilityNative
model_eval = MachineLearningEvaluateModelStabilityNative(test_records=summary)
results = model_eval.run()
true_classes = results["true_classes"] 
stability_score = results["stability_score"] 

print("True classes: ", true_classes)
print("Model stability score: ", stability_score)

if (true_classes["class_true"] == "not_set").any():
    print("Classification Complete")
else:
    print("Some samples are unverified. Setting true_classes to majority class")
    mask = true_classes["class_true"] == "not set"
    true_classes.loc[mask, "class_true"] = true_classes.loc[mask, "majority_class"]

Model performance Summary:     index  threshold     score  confidence    class  \
0     14  -0.080903 -0.111895        1.38  outlier   
1     47  -0.061153 -0.093758        1.53  outlier   
2     47  -0.061153 -0.093758        1.53  outlier   
3     49  -0.060696 -0.070741        1.17  outlier   
4     50  -0.061153 -0.016271        0.27   normal   

                         date class_true  stability_score  
0  2025-07-30 10-59-59-838221    not set         0.545455  
1  2025-07-29 16-35-41-271047    not set         1.000000  
2  2025-07-29 16-35-46-848140    not set         1.000000  
3  2025-07-29 16-40-22-832458    not set         0.545455  
4  2025-07-29 16-35-41-271047    not set         0.545455  
True classes:      index class_true majority_class  confidence_consistency
0      14    not set        outlier                0.090909
1      47    not set        outlier                1.000000
2      49    not set        outlier                0.090909
3      50    not set         nor

In [14]:
# sorted. Minor edits to ensure identical column order 
new_data["label"] = true_classes["class_true"]
col = new_data.pop("label")
new_data.insert(14, "label", col)
print("New data: ", new_data[metadata_columns])

New data:      index             name                                               path  \
0      14       210812_Pac  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
1      47       210813_Pac  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
2      49  210813_Pac--003  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
3      50  210813_Pac--004  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
4     116       210819_Pac  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
5     117  210819_Pac--002  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
6     118  210819_Pac--003  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
7     119  210819_Pac--004  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
8     120  210819_Pac--005  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
9     121  210819_Pac--006  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
10    122  210819_Pac--007  C:/Users/Sandeep/Desktop/ExtractedSignals\2108...   
11    123  210819

In [15]:
#combine all available labeled data, features and metadata included 
old_train_data = old_train_data.reset_index(drop=True)
new_data.reset_index(drop=True, inplace=True)

labeled_data = pd.concat([old_train_data, new_data], ignore_index=True)
print("KNN Training Data: ", labeled_data.shape)

# drop unclassified samples
labeled_data = labeled_data[labeled_data["label"] != "not set"]

# shuffle dataframe randomly by rows and keep 100% (frac = 1). unnecessary, train_test_split does it via shuffle=True
# labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
# print("Shuffled: ", labeled_data.shape)

metadata = labeled_data[metadata_columns]
labels = metadata["label"]
labeled_data = labeled_data.drop(columns= metadata_columns)

KNN Training Data:  (67, 32)


In [16]:
# create random splits of training and test sets. Shuffling and train-test-splitting with 'stratify' ensure a good distribution of classes so KNN learning is effective
from sklearn.model_selection import train_test_split
features_train, features_test, metadata_train, metadata_test = train_test_split(labeled_data, metadata, test_size=0.3, stratify=metadata["label"])

In [17]:

from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses
knn_ana = MachineLearningAnalyses(variables=features_train, metadata=metadata_train)

from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn
scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="StandardScaler")
knn_ana = scl.run(knn_ana)

from src.StreamPort.machine_learning.methods import MachineLearningMethodNearestNeighboursClassifierSklearn
knn = MachineLearningMethodNearestNeighboursClassifierSklearn(n_neighbors=1) #odd number for binary classification. 1 usually causes overfit, could be suitable for small amount of data
knn_ana = knn.run(knn_ana)

print(knn_ana)


NearestNeighboursAnalyses 
  variables: 46 rows, 16 columns
  metadata: 46 rows, 16 columns



In [18]:
knn_ana.plot_data()

c:\Users\Sandeep\Desktop\StreamPort\src\StreamPort\machine_learning\analyses.py:69: UserWarning:

DataFrame columns are not unique, some columns will be omitted.



In [19]:
knn_ana.train()
train_classes = knn_ana.get_training_scores()
print(train_classes)

['normal' 'normal' 'normal' 'normal' 'normal' 'normal' 'outlier' 'normal'
 'normal' 'normal' 'normal' 'normal' 'outlier' 'normal' 'normal' 'normal'
 'normal' 'normal' 'normal' 'outlier' 'normal' 'normal' 'normal' 'outlier'
 'normal' 'outlier' 'normal' 'normal' 'outlier' 'normal' 'normal' 'normal'
 'normal' 'normal' 'outlier' 'outlier' 'outlier' 'normal' 'normal'
 'normal' 'normal' 'normal' 'normal' 'outlier' 'normal' 'outlier']


In [20]:
knn_ana.predict(features_test, metadata_test)
classes = knn_ana.get_prediction_classes()
print(classes)

    index   labels
52    237  outlier
6      24   normal
29    121   normal
23     50   normal
49    167   normal
63    271   normal
0      18   normal
17     58   normal
22     49   normal
53    238  outlier
5      23   normal
58    245  outlier
55    242  outlier
66    278  outlier
62    269  outlier
11     52   normal
26    118   normal
64    272  outlier
3      21   normal
10     51   normal
7      25   normal


In [21]:
probs = knn_ana.get_prediction_probabilities()
print(probs)

[1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0]


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

classified_indices = list(true_classes["index"])
for i in range(len(classified_indices)):
    current_index = classified_indices[i]
    this_group_in_summary = summary[summary["index"] == current_index]
    print("this Index: ", current_index, " and group: ", this_group_in_summary)
    if this_group_in_summary.empty:
        continue

    # X axis is repetition count: 1, 2, ..., N
    x_vals = list(range(1, len(this_group_in_summary) + 1))

    # Map 'class' to 0/1
    y1 = this_group_in_summary["class"].map({"normal": 0, "outlier": 1}).tolist()
    y2 = this_group_in_summary["confidence"].tolist()

    # Class trace (left Y axis)
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y1,
        mode="lines+markers",
        yaxis="y1",
        marker=dict(symbol="circle", color="red"),
        line=dict(dash="dot"),
        hovertext=this_group_in_summary["class"],
        hoverinfo="text"
    ))

    # Confidence trace (right Y axis)
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y2,
        mode="lines+markers",
        yaxis="y2",
        marker=dict(symbol="square", color="blue"),
        line=dict(dash="solid"),
        hovertemplate="Confidence: %{y}<extra></extra>"
    ))

# Layout setup
fig.update_layout(
    title="True Class (0/1) and Confidence by Test Repetition",
    xaxis=dict(title="Test Repetition Number"),
    yaxis=dict(
        title="Class (0=Normal, 1=Outlier)",
        tickvals=[0, 1],
        ticktext=["Normal", "Outlier"],
        tickfont=dict(color="red")
    ),
    yaxis2=dict(
        title="Confidence Score",
        overlaying="y",
        side="right",
        range=[0.4, 1.6],
        tickfont=dict(color="blue")
    ),
    template="simple_white",
    height=500,
    width=1000,
    legend=dict(
        x=0.5,
        y=1.1,
        xanchor="center",
        yanchor="top",
        orientation="h"
    ),
    showlegend = False
)

fig.show()
